In [2]:
!unzip "/content/GTSRB (2).zip"

Η έξοδος ροής περικόπηκε στις τελευταίες 5000 γραμμές.
  inflating: GTSRB/test_modified/26/04314.png  
  inflating: GTSRB/test_modified/26/04698.png  
 extracting: GTSRB/test_modified/26/05000.png  
 extracting: GTSRB/test_modified/26/05176.png  
 extracting: GTSRB/test_modified/26/05251.png  
 extracting: GTSRB/test_modified/26/06200.png  
 extracting: GTSRB/test_modified/26/06258.png  
 extracting: GTSRB/test_modified/26/06338.png  
 extracting: GTSRB/test_modified/26/07080.png  
 extracting: GTSRB/test_modified/26/08677.png  
  inflating: GTSRB/test_modified/26/08702.png  
 extracting: GTSRB/test_modified/26/09019.png  
 extracting: GTSRB/test_modified/26/09177.png  
 extracting: GTSRB/test_modified/26/09570.png  
 extracting: GTSRB/test_modified/26/09927.png  
 extracting: GTSRB/test_modified/26/10096.png  
 extracting: GTSRB/test_modified/26/10869.png  
  inflating: GTSRB/test_modified/26/11183.png  
  inflating: GTSRB/test_modified/26/11263.png  
 extracting: GTSRB/test_modified/

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

TRAIN_DIR = "/content/GTSRB/train_modified"
TEST_DIR  = "/content/GTSRB/test_modified"

IMG_SIZE = (96, 96)
BATCH_SIZE = 32
EPOCHS = 30

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_gen = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

num_classes = train_gen.num_classes

model = Sequential([
    Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(256, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

early_stop = EarlyStopping(patience=7, restore_best_weights=True)
checkpoint = ModelCheckpoint("cnn_scratch_gtsrb_stronger.keras", save_best_only=True)

history = model.fit(
    train_gen,
    validation_data=test_gen,
    epochs=EPOCHS,
    callbacks=[early_stop, checkpoint],
    verbose=1
)

test_loss, test_acc = model.evaluate(test_gen, verbose=0)
print(f"✅ CNN από το μηδέν (96x96) – Test Accuracy: {test_acc:.4f}")


Found 3977 images belonging to 43 classes.
Found 1983 images belonging to 43 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 176s 1s/step - accuracy: 0.1206 - loss: 6.5111 - val_accuracy: 0.0434 - val_loss: 4.0858
Epoch 2/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 172s 1s/step - accuracy: 0.3001 - loss: 2.5099 - val_accuracy: 0.1831 - val_loss: 3.9625
Epoch 3/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 176s 1s/step - accuracy: 0.4141 - loss: 1.8860 - val_accuracy: 0.2506 - val_loss: 2.7999
Epoch 4/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 172s 1s/step - accuracy: 0.4999 - loss: 1.5774 - val_accuracy: 0.4624 - val_loss: 1.7313
Epoch 5/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 172s 1s/step - accuracy: 0.5704 - loss: 1.3431 - val_accuracy: 0.5819 - val_loss: 1.2631
Epoch 6/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 171s 1s/step - accuracy: 0.6272 - loss: 1.1436 - val_accuracy: 0.6757 - val_loss: 1.1321
Epoch 7/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 178s 1s/step - accuracy: 0.7292 - loss: 0.8191 - val_accuracy: 0.7277 - val_loss: 0.9120
Epoch 8/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 172s 1s/step - accuracy: 0.7662 - loss: 0.6768 - val_accu